# EXPLAIN and EXPLAIN QUERY PLAN

## 🧠 What is EXPLAIN?

SQLite provides two commands to understand how your queries will be executed:

* **EXPLAIN QUERY PLAN:** Gives a high-level overview of how a query will be run
* **EXPLAIN:** Provides a **low-level**, detailed breakdown of the query execution plan

In most cases, you will use `EXPLAIN QUERY PLAN` for query optimization and teaching.

`EXPLAIN` is more technical and useful if you’re digging into SQLite internals.


## 🧱 Step 1: Create a Table and Populate It

We’ll create a small sales table with some customer purchase data to use in our examples.

In [1]:
import sqlite3
import random
from datetime import datetime, timedelta

# Start a fresh in-memory database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create a simple table for purchases
cursor.execute('''
CREATE TABLE sales (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer TEXT,
    product TEXT,
    sale_date TEXT,
    amount REAL
)
''')

# Fill the table with 500 sample records
customers = ['Alice', 'Bob', 'Carol', 'David', 'Eve']
products = ['Board Game', 'Dice Set', 'Card Sleeves']

for _ in range(500):
    customer = random.choice(customers)
    product = random.choice(products)
    days_ago = random.randint(0, 100)
    date = (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
    amount = round(random.uniform(10, 80), 2)
    cursor.execute("INSERT INTO sales (customer, product, sale_date, amount) VALUES (?, ?, ?, ?)",
                   (customer, product, date, amount))

conn.commit()

✅ Main point: You should understand how indexes affect performance. But before we create any indexes, let’s see how SQLite plans queries with no help.

## 🔍 Step 2: Run a Query and Inspect the Plan (No Index Yet)

We’ll inspect a query that searches for rows by customer. First, we run the query as usual:

In [2]:
query = "SELECT * FROM sales WHERE customer = 'Bob'"
cursor.execute(query)
cursor.fetchall()

[(3, 'Bob', 'Dice Set', '2025-03-17', 24.32),
 (9, 'Bob', 'Board Game', '2025-04-12', 49.02),
 (10, 'Bob', 'Board Game', '2025-03-15', 40.46),
 (12, 'Bob', 'Board Game', '2025-02-18', 46.52),
 (13, 'Bob', 'Dice Set', '2025-02-05', 48.19),
 (20, 'Bob', 'Card Sleeves', '2025-04-07', 16.78),
 (23, 'Bob', 'Dice Set', '2025-03-15', 12.9),
 (25, 'Bob', 'Card Sleeves', '2025-01-14', 75.7),
 (40, 'Bob', 'Dice Set', '2025-04-12', 25.18),
 (43, 'Bob', 'Board Game', '2025-03-21', 10.84),
 (50, 'Bob', 'Board Game', '2025-02-23', 51.76),
 (53, 'Bob', 'Dice Set', '2025-04-08', 11.04),
 (56, 'Bob', 'Dice Set', '2025-04-14', 12.3),
 (57, 'Bob', 'Dice Set', '2025-03-06', 21.15),
 (72, 'Bob', 'Board Game', '2025-02-23', 12.29),
 (82, 'Bob', 'Card Sleeves', '2025-03-27', 68.14),
 (83, 'Bob', 'Dice Set', '2025-01-07', 67.08),
 (84, 'Bob', 'Dice Set', '2025-04-13', 65.22),
 (85, 'Bob', 'Board Game', '2025-03-09', 47.71),
 (90, 'Bob', 'Card Sleeves', '2025-04-06', 42.48),
 (93, 'Bob', 'Card Sleeves', '2025-

Now, let’s use `EXPLAIN QUERY PLAN` to see how SQLite will execute that same query:

In [3]:
cursor.execute(f"EXPLAIN QUERY PLAN {query}")
for row in cursor.fetchall():
    print(row)

(2, 0, 216, 'SCAN sales')


This means SQLite is doing a full table scan — it has to check every row to find customers named “Bob”.

### 👀 Interpretation

Output format:

```
(<selectid>, <order>, <from>, <detail>)
```

* **selectid:** ID of the `SELECT` query (0 for main query, >0 for subqueries)
* **order:** Evaluation order of the `FROM` clause elements
* **from:** Table index in the `FROM` clause (0 if only one table)
* **detail:** How SQLite will access the data: scan, index, etc.


### 🧩 Step 3: Create an Index to Improve the Plan

Let’s add an index on the customer column and rerun the plan.

In [4]:
cursor.execute("CREATE INDEX idx_customer ON sales(customer)")
conn.commit()

Now run the same EXPLAIN QUERY PLAN:

In [5]:
cursor.execute(f"EXPLAIN QUERY PLAN {query}")
for row in cursor.fetchall():
    print(row)

(3, 0, 62, 'SEARCH sales USING INDEX idx_customer (customer=?)')


Now SQLite knows how the customer values are structured (via the index), it avoids the table scan. This is a key optimization concept.

### 🛠️ Step 4: Use EXPLAIN to See Bytecode

Let’s now explore what `EXPLAIN` does. This is more advanced and shows what SQLite will do step-by-step *internally* when running a query.

In [6]:
cursor.execute(f"EXPLAIN {query}")
bytecode = cursor.fetchall()

# Print first 10 instructions
for row in bytecode[:10]:
    print(row)

(0, 'Init', 0, 16, 0, None, 0, None)
(1, 'OpenRead', 0, 2, 0, '5', 0, None)
(2, 'OpenRead', 1, 10, 0, 'k(2,,)', 2, None)
(3, 'String8', 0, 1, 0, 'Bob', 0, None)
(4, 'SeekGE', 1, 15, 1, '1', 0, None)
(5, 'IdxGT', 1, 15, 1, '1', 0, None)
(6, 'DeferredSeek', 1, 0, 0, None, 0, None)
(7, 'IdxRowid', 1, 2, 0, None, 0, None)
(8, 'Column', 1, 0, 3, None, 0, None)
(9, 'Column', 0, 2, 4, None, 0, None)


**🧠 What does this output mean?**

```
(addr, opcode, p1, p2, p3, p4, p5)
```

* **addr:** Address of the instruction in the bytecode
* **opcode:** The operation to be performed (e.g., `OpenRead`, `Column`, `SeekGE`, etc.)
* **p1-p5:** Operands used by the opcode -- meaning depends on the operation
* **p4:** Extra text info (e.g., values, column names)

**🧾 Example breakdown**

```
(0, 'Init', 0, 10, 0, '', '')
(1, 'OpenRead', 0, 2, -1, 'sales', '')
(2, 'String8', 0, 1, 0, 'Bob', '')
(3, 'SeekGE', 0, 5, 1, '', '')
```

* **Init:** Starts the program
* **OpenRead:** Opens the sales table for reading
* **String8:** Loads the string 'Bob' into a register
* **SeekGE:** Moves to the first row with customer >= 'Bob' using the index (if exists)

🔬 This is how SQLite interprets the SQL into a series of internal instructions.

Should you use this? Maybe. But this is too advanced.


## 🧠 SQLite Query Planning Summary

EXPLAIN QUERY PLAN

- ✅ Shows whether SQLite will scan the table or use an index
- ✅ Easy to interpret
- ✅ Best tool for query optimization and teaching

EXPLAIN

- 🧠 Shows virtual machine (VM) bytecode steps
- ❌ Not beginner-friendly
- 🧪 Useful for internal debugging or performance analysis

**💡 Key Insight:** Indexes change how SQLite executes queries. Use EXPLAIN QUERY PLAN to confirm your queries are efficient!
